In [1]:
### EER FACTOR DAILY GENERATOR (TO BE IGNORED IN PRODUCT CODE)

In [2]:
### MODULES IMPORT (PART OF THE PRODUCT CODE)

import pandas as pd
import numpy as np
from datetime import date, datetime
import math

In [3]:
### INTERNAL PARAMETERS INITIALIZATION (TO BE IGNORED IN PRODUCT CODE)

### Universe path:
str_path_universe = 'Data_Files/Source_Files/acadian_universe.xlsx'
### Bloomberg structured data extraction keys:
str_path_bb_hdf = 'Data_Files/Source_Files/Bloomberg_prepared.h5'
str_key_fx_country = 'bb_fx_country'
str_key_fx_demeaned = 'bb_fx_demeaned'
str_key_mcap = 'bb_mcap'
str_key_reer = 'bb_reer'
str_key_neer = 'bb_neer'
str_key_reer_sourced = 'bb_reer_sourced'
str_key_neer_sourced = 'bb_neer_sourced'
str_key_export_monthly = 'bb_export'
str_key_gdp = 'bb_gdp'
str_key_cpi = 'bb_cpi'
### General daily-mone ranges parameters:
str_source_date_start = '1992-01-01' ### Start date for source vectors
str_measure_date_start = '1996-08-01' ### Start date for efficacy measures
str_ison_date_start = '1994-01-31' ### Start date for ISON Universe
str_measure_date_end = '2020-08-31' ### End date for efficacy measures
idx_source_date_range = pd.date_range(str_source_date_start, str_measure_date_end, freq = 'B') ### Range for source data filtering
idx_test_monthly_date_range = pd.date_range(str_ison_date_start, str_measure_date_end, freq = 'BM') ### Range for source data filtering
idx_test_daily_date_range = pd.date_range(str_ison_date_start, str_measure_date_end, freq = 'B') ### Range for source data filtering
idx_factor_date_range = pd.date_range(str_source_date_start, str_measure_date_end, freq = 'BM') ### Range for factor data filtering
idx_measure_date_range = pd.date_range(str_measure_date_start, str_measure_date_end, freq = 'BM') ### Range for measures calculation

In [4]:
### GENERAL PARAMETERS INITIALIZATION (PART OF THE PRODUCT CODE)

### Common constants:
All = slice(None)
date_source_start = pd.to_datetime('1992-01-01') ### Start date for source vectors

### Standartization parameters:
list_truncate = [2.5, 2.0] ### Standartization boundaries
bool_within_market = True ### Standartization option

### ISON filtering options:
list_ison = ['DM', 'EM', 'FM'] ### Regions filter to drop NaN region values
list_countries_to_exclude = ['VE'] ### Countries not to play the game

In [5]:
### DEFINING MATLAB STYLE PRCTILE (PART OF THE PRODUCT CODE)

def prctile_matlab(ser_to_perc, p):
    ### Sorted list preparing:
    list_to_perc = ser_to_perc.dropna().values
    list_sorted = np.sort(list_to_perc)
    ### Length calculating:
    num_len = len(list_to_perc)    
    ### Prctile calculating:
    num_result = np.interp(np.array(p), np.linspace(1 / (2 * num_len), (2 * num_len - 1) / (2 * num_len), num_len), list_sorted)
    ### Results output:
    return num_result

In [6]:
### DEFINING EXPONENTIAL WEIGHT (PART OF THE PRODUCT CODE)

def exp_weight_single(halflife_len = 3, num_element = 0):
    ### Weight calculating:
    num_period_factor = math.exp(math.log(0.5) / round(halflife_len))
    num_weight = np.exp(math.log(num_period_factor) * num_element)
    ### Result output:
    return num_weight

In [7]:
### DEFINING GEOMETRICAL WEIGHT (PART OF THE PRODUCT CODE)

def geom_weight_single(flo_ratio, flo_factor = 1, num_element = 0):
    ### Results output:
    return flo_factor * (flo_ratio ** num_element)

In [8]:
### DEFINING WEIGHTED AVERAGE (PART OF THE PRODUCT CODE)

def weighted_average(ser_data, ser_weight = False, int_min_count = 0):
    ### Default output:
    num_result = np.NaN
    ### Checking for data presence:
    if (ser_data.count() > int_min_count):       
        ### Checking for weights dataset:
        if isinstance(ser_weight, bool):
            ### Calculating of simple average:
            num_result = np.nanmean(ser_data.values)
        else:
            ### Weights filtering:
            list_weight = ser_weight[ser_data.dropna().index].values
            ### Checking for weights presence:
            if np.nansum(list_weight):
                ### Data filtering:
                list_data = ser_data.dropna().values
                ### Weighted average calculating:
                num_result = np.nansum(list_data * list_weight) / np.nansum(list_weight)
    ### Results output:
    return num_result

In [9]:
### DEFINING WEIGHTED AVERAGE FOR DATAFRAME COLUMNS (PART OF THE PRODUCT CODE)

def columns_average(df_series, list_weights = False):
    ### Equal weights list creating:
    if isinstance(list_weights, bool):
        list_weights = [1] * len(df_series.columns)
    ### Dataframe of weights initialising:
    df_weights = pd.DataFrame([list_weights] * len(df_series.index), index = df_series.index, columns = df_series.columns)
    ### Zeroing weights for NaN values:
    for iter_col in df_weights.columns:
        df_weights.loc[df_series[iter_col].isna(), iter_col] = 0
    ### Weighted mean calulating:
    df_means = (df_series * df_weights).sum(axis = 1) / df_weights.sum(axis = 1)    
    ### Results output:
    return df_means

In [10]:
### DEFINING MULTI-STEP STANDARTIZATION FOR SEPARATE SERIES (PART OF THE PRODUCT CODE)

def multistep_standartize(ser_data_source, arr_truncate, ser_weight = False, reuse_outliers = False, center_result = True, full_result = False):  
    ### Arrays of iterations properties:
    arr_mean = []
    arr_std = []
    ### Adding equal weights, when weights are absent:
    if isinstance(ser_weight, bool):
        ser_weight = pd.Series(1, index = ser_data_source.index)
        ser_weight.name = 'Weight'    
    ### Workhorse and resulting data vectors initialising:
    ser_data_iter = ser_data_source.dropna()
    ser_weight_iter = ser_weight.copy()
    ser_data_full = pd.Series(np.NaN, index = ser_data_iter.index)
    ### Looping by boundaries array:
    for num_bound_iter in arr_truncate:
        ### Properties calculating and saving:
        num_mean_iter = weighted_average(ser_data_iter, ser_weight_iter)
        num_std_iter = ser_data_iter.std()
        arr_mean.append(num_mean_iter)
        arr_std.append(num_std_iter)
        ser_data_iter = (ser_data_iter - num_mean_iter) / num_std_iter       
        ### Standartizing:
        if reuse_outliers:
            ser_data_iter[ser_data_iter.abs() >= num_bound_iter] = np.sign(ser_data_iter) * num_bound_iter 
        else:
            ### Saving to result and excluding from further calculations truncated values:             
            ser_data_full.where(ser_data_iter.abs() < num_bound_iter, np.sign(ser_data_iter) * num_bound_iter, inplace = True)
            ser_data_iter = ser_data_iter[ser_data_iter.abs() < num_bound_iter]           
    ### Aggregating result:
    if (reuse_outliers):
        ser_data_full = ser_data_iter
    else:     
        ser_data_full[ser_data_iter.index] = ser_data_iter
    ### Centering result:
    if (center_result):
        ser_result = ser_data_full - weighted_average(ser_data_full, ser_weight) 
    else:
        ser_result = ser_data_full    
    ### Result output:
    ser_result.name = str(ser_data_source.name) + '_standartized'
    if (full_result):
        return (ser_result, arr_mean, arr_std)
    else:
        return ser_result

In [11]:
### DEFINING MULTI-STEP STANDARTIZATION BY MARKET FOR CROSS-SECTION (PART OF THE PRODUCT CODE)

def ison_standartize(ser_to_manage, arr_truncate, ser_weight = False, reuse_outliers = False, center_result = True, full_result = False, within_market = False):
    ### Multi-step standartizing:
    if (within_market):
    ### Within market standartizing:
        ser_result = ser_to_manage.groupby(by = 'Market', group_keys = False).apply(multistep_standartize, arr_truncate, ser_weight, 
                                                                                                  reuse_outliers, center_result, full_result)
    else:
    ### Full universe standartizing:
        ser_result = multistep_standartize(ser_to_manage, arr_truncate, ser_weight, reuse_outliers, center_result, full_result)
    ### Results output:
    return ser_result

In [12]:
# DEFINING MULTI-STEP STANDARTIZATION BY MARKET FOR FULL FACTOR STACK (PART OF THE PRODUCT CODE)

def single_factor_standartize_daily(ser_factor, arr_truncate, ser_weight = False, reuse_outliers = False, center_result = True, within_market = False):
    ### Weights preparing:
    if isinstance(ser_weight, bool):
        ser_weight = pd.Series(1, index = ser_factor.index)
        ser_weight.name = 'Weight'
    ### Multi-step standartizing:        
    df_factor = ser_factor.to_frame().join(ser_weight, how = 'left')
    df_factor.columns = ['Factor', 'Weight']
    ser_result = ison_standartize(df_factor['Factor'], arr_truncate, df_factor['Weight'], reuse_outliers, center_result, False, within_market)
    ### Results output:
    ser_result.name = ser_factor.name
    return ser_result   

In [13]:
### DEFINING EXTRACTION UNIVERSE DATA FROM MS EXCEL SOURCE (TO BE IGNORED IN PRODUCT CODE)

def ison_membership_converting(str_path_universe, date_end, bool_daily = False, int_backfill_months = 0):
    ### Defining business-month-end reindexation on country level:
    def country_modify(ser_raw_country, date_end):
        ser_res_country = ser_raw_country.droplevel(0).resample('MS').last().resample('BM').last()
        range_country = pd.date_range(ser_res_country.index[0], date_end, freq = 'BM')
        return ser_res_country.reindex(range_country).ffill()
    ### Markets encoding table:
    dict_markets = {50 : 'DM', 57 : 'EM', 504 : 'FM', 0: np.NaN}     
    ### Loading source file:
    df_raw_universe = pd.read_excel(io = str_path_universe, sheet_name = 0, header = 0, parse_dates = True, index_col = [0, 1],
                                 na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', 
                                             '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null'], keep_default_na = False)
    ### Converting source file:
    df_raw_universe.index.names = ['Country', 'Date']
    ser_raw_universe = df_raw_universe['Region']
    ser_raw_universe.fillna(0, inplace = True)
    ser_raw_universe.name = 'Market'
    ### By country reindexation and translation:
    ser_res_universe = ser_raw_universe.groupby('Country').apply(country_modify, date_end)
    ser_res_universe.index.names = ['Country', 'Date']
    ser_res_universe = ser_res_universe.replace(dict_markets).reorder_levels([1, 0]).sort_index() 
    ### Expanding membership for primary regions members by backfilling:
    if int_backfill_months:
        ### List of regions:
        list_region = list(ser_res_universe.dropna().unique())
        ### Initialising of collection of series with backfilled data for each region:
        list_ison_backfill = []
        ### Regions looping:
        for iter_region in list_region:
            ### Defining start of region date:
            date_first_valid = ser_res_universe.loc[ser_res_universe == iter_region].first_valid_index()[0]
            ### Creating dates index to backfilling:
            idx_date_backfill = pd.date_range(end = date_first_valid, periods = int_backfill_months + 1, freq = 'BM')[: -1]
            ### Creating primary countries index to backfilling:            
            idx_region_backfill = ser_res_universe.loc[ser_res_universe == iter_region].loc[date_first_valid, All].index.get_level_values('Country')
            ### Creating full index:
            idx_ison_backfill = pd.MultiIndex.from_product([idx_date_backfill, idx_region_backfill])
            ### Series with backfilled data:
            list_ison_backfill.append(pd.Series(iter_region, index = idx_ison_backfill))
        ### Combination of backfilled series and original ISON data:    
        ser_res_universe = ser_res_universe.combine_first(pd.concat(list_ison_backfill, axis = 0)).sort_index()  
        ser_res_universe.index.names = ['Date', 'Country']
    ### Converting to daily frequency:
    if bool_daily:
        ser_res_universe = ser_res_universe.reset_index('Country').groupby('Country').resample('B').ffill()['Market'].swaplevel().sort_index()    
    ### Results output:
    ser_res_universe.name = 'Market'
    return ser_res_universe

In [14]:
### DEFINING MEAN MOMENTUM FUNCTION (PART OF THE PRODUCT CODE)

def rolling_weighted_mean(ser_country_source, int_mean_win, int_mean_min, list_weight):
    try:
        ### Weight setting
        ser_weight = pd.Series(list_weight[ : len(ser_country_source.index)], ser_country_source.index)  
        ### Weighted mean calculation:
        return weighted_average(ser_country_source, ser_weight, int_mean_min)
    except KeyError:
        return np.NaN

In [15]:
### DEFINING FILTERING DATE INTERVAL, REINDEXING FILTERED VECTOR TO BUSINESS DATES/MONTHS FREQUENCY AND FILLING DATA

def get_country_interval(ser_filtered, date_start, date_end, int_fill_limit = 1):
    ### Business day filter:
    idx_date_business = pd.date_range(start = date_start, end = date_end, freq = 'B')
    try:
        ### Frequency checker:
        date_first = ser_filtered.first_valid_index()[0]
        date_last = ser_filtered.last_valid_index()[0]
        ### Resampling to business month:
        if ((date_last - date_first).days / len(ser_filtered.dropna().index) > 3.0):          
            ser_filtered = ser_filtered.unstack('Country').resample('MS').last().resample('BM').last().stack('Country')
    except TypeError:
        pass
    finally:
        ### Reindexation and forward filling:
        ser_reindexed = ser_filtered.unstack('Country').resample('B').ffill().fillna(method = 'ffill', limit = int_fill_limit)\
                                                       .reindex(idx_date_business).ffill(limit = int_fill_limit).stack('Country', dropna = False)        
        ### Results output:
        ser_reindexed.index.names = ['Date', 'Country']        
    return ser_reindexed

In [16]:
### DATA LOADING (TO BE IGNORED IN PRODUCT CODE)

ser_reer = pd.read_hdf(str_path_bb_hdf, key = str_key_reer_sourced) ### Real Effective Exchange Rate to use as a factor data source
ser_ison_daily = ison_membership_converting(str_path_universe, datetime.strptime(str_measure_date_end, '%Y-%m-%d'), bool_daily = True) ### ISON universe, bus-daily vector

In [17]:
### TESTING: LONG-TERM REER BASED FACTOR CALCULATION: DIRECT DAILY SCRIPT SINGLE RUN FOR DEBUGGING (TO BE IGNORED IN PRODUCT CODE)

iter_date = idx_test_daily_date_range[580] # '1996-04-22'
iter_date = pd.to_datetime('2006-12-29')
#iter_date = idx_test_daily_date_range[6586] # '2019-04-30'

### Momentum parameters:
int_mom_hl = 520 ### Without rounding here
int_mom_win = 1300
int_mom_min = 520
### Weights array:
list_weight = list(map(lambda iter_num: exp_weight_single(int_mom_hl, iter_num), range(int_mom_win)))[::-1]
### Source loading:
int_fill_limit = 66
date_start_win = np.maximum(iter_date - pd.tseries.offsets.BDay(int_mom_win - 1), date_source_start)
date_start_loc = np.maximum(iter_date - pd.tseries.offsets.BDay(int_mom_win + int_fill_limit), date_source_start)
### Datasource for particular date (should be substituted by SQL query):
ser_iter_source_raw = ser_reer.droplevel('Source').loc[pd.date_range(start = date_start_loc, end = iter_date, freq = 'D'), All]
### Data source resampling:
ser_iter_source = ser_iter_source_raw.groupby('Country', group_keys = False)\
                                     .apply(get_country_interval, date_start_win, iter_date, int_fill_limit)
### Source performing:
ser_iter_delta = ser_iter_source.groupby('Country').diff() / ser_iter_source.groupby('Country').shift()   
ser_iter_delta = ser_iter_delta.replace([np.inf, -np.inf], np.NaN)    
ser_iter_delta.index.names = ['Date', 'Country']
### Momentum factor calculation:
ser_iter_factor = ser_iter_delta.groupby('Country').apply(rolling_weighted_mean, int_mom_win, int_mom_min, list_weight)
### Date index recreating:
ser_iter_factor = pd.concat({iter_date: ser_iter_factor}, names = ['Date'])
### ISON Universe for the date loading (should be substituted by SQL query):
ser_ison_iter_date = ser_ison_iter_date = ser_ison_daily.loc[iter_date, All]
### Factor ISONing:
ser_iter_factor = ser_iter_factor.to_frame().join(ser_ison_iter_date, how = 'left').set_index('Market', append = True).squeeze()
ser_iter_factor.name = 'Factor' 
### Regions clearing:
ser_iter_factor = ser_iter_factor.loc[All, All, list_ison]
### Countries filtering:
ser_iter_factor = ser_iter_factor.drop(list_countries_to_exclude, level = 'Country')    
### Standalone factor standartizing and sign changing:
ser_iter_factor_std = -single_factor_standartize_daily(ser_iter_factor, list_truncate, within_market = bool_within_market)
ser_iter_factor_std.name = 'Factor'        
#### Interaction variable applying:
#ser_iter_multiplied = ser_iter_factor_std * ser_iter_date_concept
#### Multiplied factor restandartizing:
#ser_iter_multiplied_std = single_factor_standartize_daily(ser_iter_multiplied, list_truncate, within_market = bool_within_market)
#ser_iter_multiplied_std.name = 'Factor'    
#### Saving result to collection:
#dict_iter_factor[iter_factor] = ser_iter_multiplied_std

In [31]:
### TESTING: LONG-TERM REER BASED FACTOR CALCULATION: DIRECT DAILY SCRIPT SINGLE RUN FOR DEBUGGING (TO BE IGNORED IN PRODUCT CODE)

#ser_iter_factor.loc[All, 'JO']
#ser_iter_factor_std.abs().sum()

In [18]:
### DEFINING FACTOR CREATING FUNCTION (PART OF THE PRODUCT CODE)

def get_lt_reer_factor(iter_date):
    ### Momentum parameters:
    int_mom_hl = 520 ### Without rounding here
    int_mom_win = 1300
    int_mom_min = 520
    ### Weights array:
    list_weight = list(map(lambda iter_num: exp_weight_single(int_mom_hl, iter_num), range(int_mom_win)))[::-1]
    ### Source loading:
    int_fill_limit = 66
    date_start_win = np.maximum(iter_date - pd.tseries.offsets.BDay(int_mom_win - 1), date_source_start)
    date_start_loc = np.maximum(iter_date - pd.tseries.offsets.BDay(int_mom_win + int_fill_limit), date_source_start)
    ### Datasource for particular date (should be substituted by SQL query):
    ser_iter_source_raw = ser_reer.droplevel('Source').loc[pd.date_range(start = date_start_loc, end = iter_date, freq = 'D'), All]
    ### Data source resampling:
    ser_iter_source = ser_iter_source_raw.groupby('Country', group_keys = False)\
                                         .apply(get_country_interval, date_start_win, iter_date, int_fill_limit)
    ### Source performing:
    ser_iter_delta = ser_iter_source.groupby('Country').diff() / ser_iter_source.groupby('Country').shift()   
    ser_iter_delta = ser_iter_delta.replace([np.inf, -np.inf], np.NaN)    
    ser_iter_delta.index.names = ['Date', 'Country']
    ### Momentum factor calculation:
    ser_iter_factor = ser_iter_delta.groupby('Country').apply(rolling_weighted_mean, int_mom_win, int_mom_min, list_weight)
    ### Date index recreating:
    ser_iter_factor = pd.concat({iter_date: ser_iter_factor}, names = ['Date'])
    ### ISON Universe for the date loading (should be substituted by SQL query):
    ser_ison_iter_date = ser_ison_iter_date = ser_ison_daily.loc[iter_date, All]
    ### Factor ISONing:
    ser_iter_factor = ser_iter_factor.to_frame().join(ser_ison_iter_date, how = 'left').set_index('Market', append = True).squeeze()
    ser_iter_factor.name = 'Factor' 
    ### Regions clearing:
    ser_iter_factor = ser_iter_factor.loc[All, All, list_ison]
    ### Countries filtering:
    ser_iter_factor = ser_iter_factor.drop(list_countries_to_exclude, level = 'Country')
    ### Standalone factor standartizing and sign changing:
    ser_iter_factor_std = -single_factor_standartize_daily(ser_iter_factor, list_truncate, within_market = bool_within_market)
    ser_iter_factor_std.name = 'Factor'      
    ### Results output:
    return ser_iter_factor_std

In [ ]:
### TESTING: PERFORMING FACTOR FOR DATE RANGE (TO BE IGNORED IN PRODUCT CODE)

#iter_date = idx_test_monthly_date_range[580]
#iter_date = idx_test_monthly_date_range[586]
#print(iter_date)

### Local testing parameters:
int_interval = 10
dict_factor_by_date = {}
date_start = datetime.utcnow()
date_control = datetime.utcnow()
idx_test_date_range = idx_test_monthly_date_range # idx_test_monthly_date_range[311 : 320] # 
### Test performing:
print('Start time:', date_start)
for iter_num, iter_date in enumerate(idx_test_date_range):
    if not (divmod(iter_num, int_interval)[1]):
        if iter_num:
            print('Counter marker:', iter_num, '/', len(idx_test_date_range))
            timedelta_interval = datetime.utcnow() - date_control
            print('Time interval since last marker:', datetime.utcnow() - date_control)            
            print('Average interval for single date:', str(timedelta_interval / int_interval))
        date_control = datetime.utcnow()
    dict_factor_by_date[iter_date] = get_lt_reer_factor(iter_date)
date_finish = datetime.utcnow()
print('Finish time:', date_finish)
print('Full interval:', date_finish - date_start)
print('Average interval for single date:', str((date_finish - date_start) / len(idx_test_date_range)))
ser_factor_full = pd.concat(dict_factor_by_date, axis = 0).droplevel(0).sort_index()
ser_factor_full.name = 'Combo Factor'

In [24]:
### TESTING: PERFORMING FACTOR FOR DATE RANGE (TO BE IGNORED IN PRODUCT CODE)

ser_factor_full.to_excel('Data_Files\Test_Files\Acadian_mode.xlsx', merge_cells = False)